# Assignment 1

In [5]:
import re
import helper
import numpy as np
import pandas as pd

ODI_DATASET_PATH = './data/ODI-2022.csv'

In [6]:
# # load csv file to pandas dataframe to retrieve column names
# df_temp= pd.read_csv("./data/ODI-2022.csv", delimiter=";")
# df_temp

In [7]:
# Initialize 2 dimensional array
dataset = helper.get_data(ODI_DATASET_PATH)
# Initialize columns
columns = list(zip(*dataset))

# print("What programme are you in? = {}".format(columns[1]))
# print("Have you taken a course on machine learning? = {}".format(columns[2]))
# print("Have you taken a course on information retrieval?; = {}".format(columns[3]))
# # mu is yes, sigma is no
# print("Have you taken a course on statistics? = {}".format(columns[4]))
# print("Have you taken a course on databases? = {}".format(columns[5]))
# print("What is your gender? = {}".format(columns[6]))
# print("Chocolate makes you..... = {}".format(columns[7]))
# print("When is your birthday (date)? = {}".format(columns[8]))
# print("Number of neighbors sitting around you? = {}".format(columns[9]))
# print("Did you stand up? = {}".format(columns[10]))
# print("What is your stress level (0-100)? = {}".format(columns[11]))
# print("You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then? = {}".format(columns[12]))
# print("Give a random number = {}".format(columns[13]))
# print("Time you went to be Yesterday = {}".format(columns[14]))
# print("What makes a good day for you (1)?; = {}".format(columns[15]))
# print("What makes a good day for you (2)? = {}".format(columns[16]))



In [21]:
# load csv file to pandas dataframe to retrieve column names
df_temp= pd.read_csv("./data/ODI-2022.csv", delimiter=";")
column_names = df_temp.columns

# convert dataset to pandas dataframe using column names from df_temp
df = pd.DataFrame(dataset)
df.columns=column_names

df_temp = df_temp.drop(labels=148, axis=0)
df_temp = df_temp.drop(labels=205, axis=0)

## clean all column values
# remove unimportant columns
df = df.drop("Tijdstempel", axis=1) # remove Tijdstempel column because it has no meaning
df = df.drop("Did you stand up?", axis=1) # remove stand up column because it has no meaning
df = df.drop("Number of neighbors sitting around you?", axis=1) # remove neighbours column because it has no meaning
df = df.drop("Give a random number", axis=1) # remove random number column because it has no meaning

# convert column values to boolean for yes/no questions
df['Have you taken a course on machine learning?'] = df['Have you taken a course on machine learning?'].replace(['yes','no','unknown'],[True,False,False])
df['Have you taken a course on information retrieval?'] = df['Have you taken a course on information retrieval?'].replace([1,0,-1],[True,False, False])
df['Have you taken a course on statistics?'] = df['Have you taken a course on statistics?'].replace(['mu','sigma', 'unknown'],[True,False,False])
df['Have you taken a course on databases?'] = df['Have you taken a course on databases?'].replace(['ja','nee', 'unknown'],[True,False,False])

# only keep years in birthday column
def get_year(date):
    year = re.search('[0-9]{4}', date)
    symbols = {'/', '.', '-'}
    if year == None:
        if len(date) > 4 and any(sym in date for sym in symbols):
            year = "19" + date[-2:]
            if year.isdecimal():
                year = int(year)
                if 1930 > year:
                    return np.nan
                elif year > 2005:
                    return np.nan
                else:
                    return year
            else:
                return np.nan
        else:
            return np.nan
    else:
        year = int(year.group(0))
        if 1930 > year:
            return np.nan
        elif year > 2005:
            return np.nan
        else:
            return year

# iterate all rows of birthday column and set all birthdays to year
for index, row in df.iterrows():
    birthday = (row["When is your birthday (date)?"])
    new_birthday = get_year(birthday)
    df['When is your birthday (date)?'] = df['When is your birthday (date)?'].replace([birthday],new_birthday)

# clean stress level column --> set all values < 0 to 0 and > 100 to 100
df = df.astype({'What is your stress level (0-100)?': int}, errors='raise')
df['What is your stress level (0-100)?'].values[df['What is your stress level (0-100)?'].values > 100] = 100
df['What is your stress level (0-100)?'].values[df['What is your stress level (0-100)?'].values < 0] = 0

# clean competition column --> set all values < 0 to 0 and > 100 to 100
df.rename(columns = {'You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then? ':'Competition'}, inplace = True)
df = df.astype({'Competition': int}, errors='raise')
df['Competition'].values[df['Competition'].values > 100] = 100
df['Competition'].values[df['Competition'].values < 0] = 0

# clean bedtime column
df.rename(columns={'Time you went to be Yesterday': 'Bedtime'}, inplace=True)

def clean_time(old):
    symbols = {'am', 'pm'}
    old = old.lower()
    if re.match('^[0-9]{2}\:[0-9]{2}$', old):
        return old
    elif re.match('^[0-9]{2}\:[0-9]{2}$', old[0:5]):
        return old[0:5]
    elif re.match('^[0-9]{2}$', old):
        keep = {'9', '10', '11', '12', '21', '22', '23', '24'}
        if old in keep:
            return old + ':' + '00'
        else:
            return np.nan
    elif re.match('^[0-9]{2}\.[0-9]{2}$', old):
        return old[0:2] + ':' + old[3:5]
    elif re.match('^[0-9]{1}\.[0-9]{2}$', old):
        return '0' + old[0] + ':' + old[2:4]
    elif re.match('^[0-9]{1}\:[0-9]{2}$', old):
        return '0' + old
    elif len(old) == 1:
        return np.nan
    elif re.match('^[0-9]{4}$', old):
        return old[0:2] + ':' + old[2:4]
    elif any(sym in old[-2:] for sym in symbols):
        digits = re.sub('[^0-9]', '', old)
        if len(digits) == 4:
            return digits[0:2] + ':' + digits[2:4]
        elif len(digits) == 3:
            return '0' + digits[0] + ':' + digits[1:3]
        elif len(digits) == 2:
            if int(digits) > 9:
                return digits + ':' + '00'
            else:
                return '00' + ':' + digits
        elif len(digits) == 1:
            return '0' + digits + ':00'
        else:
            return np.nan
    else:
        return np.nan

for index, row in df.iterrows():
    old_time = (row['Bedtime'])
    new_time = clean_time(old_time)
    df['Bedtime'] = df['Bedtime'].replace([old_time], new_time)

# clean good day columns
df.rename(columns={'What makes a good day for you (1)?': 'Good day (1)'}, inplace=True)
df.rename(columns={'What makes a good day for you (2)?': 'Good day (2)'}, inplace=True)

# set1 = set(df['Good day (1)'])
# set2 = set(df['Good day (2)'])
# together = set1.union(set2)
# print(len(together))

# divide answers into categories for analyses
df.loc[df['Good day (1)'].str.contains('coffee', case=False), 'Good day (1)'] = 'Coffee'
df.loc[df['Good day (1)'].str.contains('sun|rain|weather|wind|skies', case=False), 'Good day (1)'] = 'Weather'
df.loc[df['Good day (1)'].str.contains('sleep|early|wake up late', case=False), 'Good day (1)'] = 'Sleep'
df.loc[df['Good day (1)'].str.contains('friend|family|social|people', case=False), 'Good day (1)'] = 'Social relationships'
df.loc[df['Good day (1)'].str.contains('sport|volley|gym|football|tennis|bouldering|biking|exercise|workout', case=False), 'Good day (1)'] = 'Physical activity'
df.loc[df['Good day (1)'].str.contains('food|drinks|drinking|pasta|lasagne|chcocolate|beer|cookies|meal|sushi|cake|alcohol|croissant', case=False), 'Good day (1)'] = 'Food/drinks'
df.loc[df['Good day (1)'].str.contains('mental|stress|relax|mood', case=False), 'Good day (1)'] = 'Mental wellbeing'
df.loc[df['Good day (1)'].str.contains('productiv|good work|work done|challeging works', case=False), 'Good day (1)'] = 'Productivity'



# we defined the following categories:
# weather
# social relationships
# coffee
# good sleep
# sports
# food/drinks
# mental welbeing
# productivity
# studying
# reading
# money
# drugs
# data mining techniques
# positivity
# free time


# What kinds of attributes are there
## Boolean, object, integer, float
# print(df['What is your gender?'].unique())
# print(df.dtypes)
# display(df)


# Ranges of values
# Distribution of values,
# relationships between attributes
# and so on.

392


In [6]:
# How many records are there
n_records = df.shape[0]
print(f"There are {n_records} records in the file.")

# How many attributes?
n_attributes = len(df.columns)
print(f"There are {n_attributes} attributes in the file.")

# Make various plots of the data. Is there something interesting worth reporting? Re-
# port the figures, discuss what is in them. What meaning do those bars, lines, dots, etc.
# convey? Please select essential and interesting plots for discussion, as you have limited
# space for reporting your findings (see details in a later section).